In [ ]:
!pip install yfinance gudhi matplotlib numpy pandas

import yfinance as yf
import numpy as np
import pandas as pd
import gudhi as gd
import matplotlib.pyplot as plt
from gudhi import RipsComplex, SimplexTree
from gudhi.hera import wasserstein_distance

# -------------------------------
# Data Acquisition and Preparation
# -------------------------------
sp500 = yf.download('^GSPC', start='2005-01-01', end='2023-07-30')['Close']
nasdaq = yf.download('^IXIC', start='2005-01-01', end='2023-07-30')['Close']
djia = yf.download('^DJI', start='2005-01-01', end='2023-07-30')['Close']
russell2000 = yf.download('^RUT', start='2005-01-01', end='2023-07-30')['Close']

log_returns_sp500 = np.log(sp500 / sp500.shift(1)).dropna()
log_returns_nasdaq = np.log(nasdaq / nasdaq.shift(1)).dropna()
log_returns_djia = np.log(djia / djia.shift(1)).dropna()
log_returns_russell2000 = np.log(russell2000 / russell2000.shift(1)).dropna()

# -------------------------------
# Sliding Window Approach
# -------------------------------
window_size = 50
persistence_diagrams = []
landscapes = []

# Helper Functions
def create_landscape_function(b, d):
    return lambda x: max(min(x - b, d - x, (d - b) / 2), 0)

def calculate_lp_norm(landscape, p):
    return np.sum(np.abs(landscape)**p)**(1/p)

def rate_of_change(values):
    return np.diff(values)

# Main Analysis Loop
for i in range(len(log_returns_sp500) - window_size + 1):
    window = np.column_stack((log_returns_sp500[i:i+window_size],
                              log_returns_nasdaq[i:i+window_size],
                              log_returns_djia[i:i+window_size],
                              log_returns_russell2000[i:i+window_size]))
    rips = RipsComplex(points=window, max_edge_length=10)
    simplex_tree = rips.create_simplex_tree(max_dimension=2)
    diagram = simplex_tree.persistence()
    persistence_diagrams.append(diagram)


    # Extract 1-cycles for landscape functions
    one_cycles = [pt for pt in diagram if pt[0] == 1]
    if one_cycles:
        landscape_functions = [create_landscape_function(b, d) for _, (b, d) in one_cycles]
        x_values = np.linspace(min(b for _, (b, _) in one_cycles),
                               max(d for _, (_, d) in one_cycles),
                               1000)
        landscape_values = np.maximum.reduce([np.array([f(x) for x in x_values]) for f in landscape_functions])
    else:
        x_values = np.linspace(0, 1, 1000)
        landscape_values = np.zeros_like(x_values)

    landscapes.append(landscape_values)

# Compute L^p norms
lp_norms_p1 = [calculate_lp_norm(landscape, 1) for landscape in landscapes]
lp_norms_p2 = [calculate_lp_norm(landscape, 2) for landscape in landscapes]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.3 MB/s eta 0:00:00
